In [19]:
from sklearn.ensemble import GradientBoostingClassifier
import pandas as pd
from sklearn.model_selection import KFold
import pickle
from sklearn.metrics import classification_report
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold, train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from xgboost.sklearn import XGBClassifier

In [28]:
def train_and_save_xgb():
    """Train the XGBBoost and save the model - Loads the pipeline, Retrieve the processed data       
     .. WARNING::  Be careful to not tie knots when moving the snake!       
     :param int x: The x position where move to.        
     :param int y: The y position where move to.        
    """
    pipeline = pickle.load(open('../models/pipe.pkl', 'rb'))
    train = pd.read_csv("../data/processed/application_train.csv")
    int_train = pd.read_csv("../data/interim/train_before_normalisation.csv")
    X_train = train#.iloc[:, 1:240]
    y_train = int_train.TARGET
    #X_train = train.drop('TARGET',axis=1)#pd.concat([train, test], ignore_index=True).drop("TARGET", axis=1)
    print("training the gradient booster...")
    pipeline.named_steps['clf'].fit(train, y_train)
    print("getting the best params...")
    gs = GridSearchCV(pipeline.named_steps["clf"], {"max_depth": [1, 1.3, 1.5]}, n_jobs=-1, cv=5, scoring="accuracy")
    gs.fit(train, y_train)
    print("best params : ",gs.best_params_['max_depth'])
    pipeline.set_params(clf__max_depth=gs.best_params_['max_depth'])
    print('re training the gradient booster...')
    pipeline.named_steps['clf'].fit(train, y_train)
    pickle.dump(pipeline, open('../models/pipe.pkl','wb'))

train_and_save_xgb()

training the gradient booster...
getting the best params...


c:\Users\julie.ngan\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
10 fits failed out of a total of 15.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\julie.ngan\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\julie.ngan\AppData\Local\Programs\Python\Python38\lib\site-packages\xgboost\core.py", line 620, in inner_f
    return func(**kwargs)
  File "c:\Users\julie.ngan\AppData\Local\Programs\Python\Python38\lib\site-packages\xgboost\

best params :  1
re training the gradient booster...
